# Calculating stand level measurements from tree data

For this example, we'll start with loading some tree data and then calculate stand trees per acre, basal area. 

In [ ]:
# Load packages
# Pkg.add("ForestBiometrics) # if install needed
using ForestBiometrics, CSV, DataFrames

#Load data
datapath = Pkg.dir("ForestBiometrics", "test", "data")
df=CSV.read(joinpath(datapath, "StandExam_data.csv"),nullable=false)

This is the tree data for a single stand with 17 plots taken. To simplify, a bit we'll just explain the specs here and not have to worry about a second table with cruise spec info.

The plots were variable radius, measured with a BAF 10 prism.

ForestBiometrics exports a CONST ```K``` for the conversion factor between diameter in inches and area of a circle, as well as the metric version ```KMETRIC``` for diameter in centimeters.


In [ ]:
K

KMETRIC

# Basal Area

First let's start with calculating the basal area in ft^2 for each plot, so we need to get numbers of records/plot * 10 to get BA represented by that plot. Then we can generate our stand estimate for basal area/acre by taking the average of all plots in the stand.

In [ ]:
#make some variables, so we can reference it for simplicity
BAF = 10
n_plots=17

plot_ba=by(df,:Plot_ID,df -> DataFrame(BA = size(df, 1)*BAF))

stand_ba=mean(plot_ba[:BA])

# Trees per acre

TPA for variable radius plots is a little more involved, but nothing we can't handle. 
First, we'll calculate the appropriate expansion factor for each tree record and them combine them to get our estimate for the stand.

In [ ]:
#calculate basal area of each tree record
df[:TreeBA]=(df[:DBH].^2)*K

#calculating expansion factor for each tree record
df[:TreeEXP]=BAF./df[:TreeBA]./n_plots

# sum together to get stand estimate 
stand_tpa=sum(df[:TreeEXP])